In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import mariadb
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc

In [1]:
water_df = pd.read_csv("input/water_samples.csv")

NameError: name 'pd' is not defined

In [2]:
water_df

NameError: name 'water_df' is not defined

In [4]:
_df = pd.DataFrame()
for index, row in water_df.iterrows():
    analyses = row['analysis_name'].split(',')
    sample_code = row['sample_code']
    for analysis in analyses:
        _ = pd.DataFrame(row).T
        _['analysis_name'] = analysis
        _df = pd.concat([_df, _])

_df.to_csv("output/water_single_analysis.csv")

MemoryError: Unable to allocate 17.1 MiB for an array with shape (3, 746095) and data type object

In [6]:
_df = pd.read_csv("output/water_single_analysis.csv")

In [7]:
_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,pH,8.27,NaN
1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,pH,8.25,NaN
2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,pH,7.25,NaN
3,2,2,CB012WA0052,2012-07-19 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,pH,7.25,NaN
4,3,3,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,Electrical Conductivity,0.15,mS cm -1
...,...,...,...,...,...,...,...,...
746089,508150,508150,CM003WA0113,2019-10-09 15:16:26.000,Total Coliform and Faecal E.Coli,Chlorides,23.2,ppm
746090,508151,508151,CM003WA0113,2019-10-09 15:16:26.000,Standard Drinking Water Analysis,Bicarbonate,143,ppm
746091,508151,508151,CM003WA0113,2019-10-09 15:16:26.000,Total Coliform and Faecal E.Coli,Bicarbonate,143,ppm
746092,508152,508152,CM003WA0113,2019-10-09 15:16:26.000,Standard Drinking Water Analysis,Fluorides,0.56,ppm


In [6]:
_df.to_csv("output/water_single_analysis.csv",chunksize=1000)

In [8]:
water_df = _df.copy(deep=True)

In [9]:
np.unique([ i for i in water_df.analysis_name if "Reverse" in i])

array(['Reverse Osmosis Water Use'], dtype='<U25')

In [10]:
np.unique([ i for i in water_df.analysis_name if "Dist" in i])

array([], dtype=float64)

In [11]:
np.unique([ i for i in water_df.analysis_name if "(FAO)" in i])

array(['Chemical & Heavy Metals Water Analysis (FAO) ',
       'Irrigation Water Analysis (FAO)'], dtype='<U45')

In [12]:
np.unique([ i for i in water_df.analysis_name if "Fruit" in i])

array([], dtype=float64)

In [13]:
np.unique([ i for i in water_df.analysis_name if "Reagent" in i])

array(['Reagent Water Analysis'], dtype='<U22')

In [14]:
np.unique([ i for i in water_df.analysis_name if "Tea" in i])

array(['Tea Extract Analysis'], dtype='<U20')

In [15]:
irrigation_analysis_names = np.unique([ i for i in water_df.analysis_name if "Irrigation" in i])

In [16]:
analysis_freq = water_df.analysis_name.value_counts()

In [17]:
analysis_freq = analysis_freq.loc[analysis_freq.values >= 820]

In [18]:
water_df = water_df.loc[water_df.analysis_name.isin(analysis_freq.index)]

In [19]:
water_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,pH,8.27,NaN
1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,pH,8.25,NaN
2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,pH,7.25,NaN
3,2,2,CB012WA0052,2012-07-19 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,pH,7.25,NaN
4,3,3,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,Electrical Conductivity,0.15,mS cm -1
...,...,...,...,...,...,...,...,...
746089,508150,508150,CM003WA0113,2019-10-09 15:16:26.000,Total Coliform and Faecal E.Coli,Chlorides,23.2,ppm
746090,508151,508151,CM003WA0113,2019-10-09 15:16:26.000,Standard Drinking Water Analysis,Bicarbonate,143,ppm
746091,508151,508151,CM003WA0113,2019-10-09 15:16:26.000,Total Coliform and Faecal E.Coli,Bicarbonate,143,ppm
746092,508152,508152,CM003WA0113,2019-10-09 15:16:26.000,Standard Drinking Water Analysis,Fluorides,0.56,ppm


In [20]:
water_df.loc[water_df.analysis_name.isin(irrigation_analysis_names), "analysis_name"] = "Irrigation"

In [21]:
water_df = water_df.loc[water_df.analysis_name != 'Tea Extract Analysis']

In [22]:
water_df = water_df.loc[~(water_df['result'] == "ND") ]
water_df = water_df.loc[~(water_df['result'] == "Detected") ]

In [23]:
water_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,pH,8.27,NaN
1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,pH,8.25,NaN
2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Irrigation,pH,7.25,NaN
3,2,2,CB012WA0052,2012-07-19 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,pH,7.25,NaN
4,3,3,CB012WA0052,2012-07-19 00:00:00.000,Irrigation,Electrical Conductivity,0.15,mS cm -1
...,...,...,...,...,...,...,...,...
746089,508150,508150,CM003WA0113,2019-10-09 15:16:26.000,Total Coliform and Faecal E.Coli,Chlorides,23.2,ppm
746090,508151,508151,CM003WA0113,2019-10-09 15:16:26.000,Standard Drinking Water Analysis,Bicarbonate,143,ppm
746091,508151,508151,CM003WA0113,2019-10-09 15:16:26.000,Total Coliform and Faecal E.Coli,Bicarbonate,143,ppm
746092,508152,508152,CM003WA0113,2019-10-09 15:16:26.000,Standard Drinking Water Analysis,Fluorides,0.56,ppm


In [24]:
water_df = water_df.dropna(subset="result")

In [25]:
water_df['result'] = [ float(i.replace("<","").replace(">","").strip()) for i in water_df['result']]

In [26]:
water_df['unit_name'] = [ str(i).strip() for i in water_df['unit_name'].values ]

In [27]:
water_df['chemical_name'] = [ str(i).lower().replace(" ","_").replace("(","").replace(")","").replace(".","").strip() for i in water_df['chemical_name'].values ]

In [28]:
water_df = water_df.loc[~(water_df['result'] < 0)]

In [29]:
df_ = water_df[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/water_unit_per_chemical.csv")

In [30]:
df_ = df_.reset_index()

In [31]:
df_

,analysis_name,chemical_name,unit_name,0
0,Irrigation,electrical_conductivity,mS cm -1,6435
1,Irrigation,ph,nan,6431
2,Irrigation,boron,ppm,6390
3,Irrigation,zinc,ppm,6388
4,Irrigation,copper,ppm,6384
...,...,...,...,...
3216,Chemical & Biochemical Oxygen Demand,ryznar_index,nan,1
3217,Total Coliforms and Faecal E.Coli,iron,µmol/l,1
3218,Oil & Grease,total_organic_carbon,ppm,1
3219,Total Coliforms and Faecal E.Coli,magnesium,mmol/l,1


In [32]:
crop_chem_unit_df = pd.DataFrame()
for crop in df_['analysis_name'].unique():
    print(crop)
    chems_count_ = df_.loc[df_['analysis_name'] == crop]
    print(chems_count_)
    crop_chem_unit_df = pd.DataFrame()
    for crop in df_['analysis_name'].unique():
        print(crop)
        chems_count_ = df_.loc[df_['analysis_name'] == crop]
        duplicates_chem = (chems_count_[chems_count_.duplicated(subset='chemical_name')])
        temp_crop_chem_unit_df = pd.DataFrame()
        for index,row in duplicates_chem.iterrows():
            chemical = row['chemical_name']
            chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
            unit = chems_count_2.loc[chems_count_2[0] == max(chems_count_2[0])]['unit_name'].values[0]
            _ = pd.DataFrame({"crop":[crop],"chemical_name":[chemical], "unit_name":[unit]})
            temp_crop_chem_unit_df = pd.concat([temp_crop_chem_unit_df, _])
        crop_chem_unit_df = pd.concat([crop_chem_unit_df, temp_crop_chem_unit_df])
crop_chem_unit_df.to_csv("output/water_unit_per_chemical_decision.csv")

Irrigation
     analysis_name            chemical_name   unit_name     0
0       Irrigation  electrical_conductivity    mS cm -1  6435
1       Irrigation                       ph         nan  6431
2       Irrigation                    boron         ppm  6390
3       Irrigation                     zinc         ppm  6388
4       Irrigation                   copper         ppm  6384
...            ...                      ...         ...   ...
2810    Irrigation         feacal_coliforms   mpn/100ml     1
2812    Irrigation            feacal_e_coli   cfu/100ml     1
2813    Irrigation                 fluoride         ppm     1
3019    Irrigation                 shigella  cfu/100 ml     1
3220    Irrigation                 selenium         nan     1

[153 rows x 4 columns]
Irrigation
Feacal Coliforms (MPN) and E.Coli
Basic Drinking Water Analysis
Standard Drinking Water Analysis
Reverse Osmosis Water Use
Base Titanium Water Analysis
Standard Drinking Water
Total Coliform and Faecal E.Coli
S

In [33]:
crop_chem_unit_df

,crop,chemical_name,unit_name
0,Irrigation,sar,nan
0,Irrigation,nitrate_n,ppm
0,Irrigation,potassium,ppm
0,Irrigation,sulphur,ppm
0,Irrigation,phosphorus,ppm
...,...,...,...
0,Colloidal Silica in Water,colloidal_silica,ppm
0,Water Microbiology (KEBS),total_viable_count_@37,mpn/ml
0,KS Drinking Water Standard Analysis,total_coliforms,cfu/ml
0,KS Drinking Water Standard Analysis,faecal_e_coli,cfu/ml


In [34]:
df = pd.DataFrame()
for index, row in crop_chem_unit_df.iterrows():
    crop = row['crop']
    chemical = row['chemical_name']
    unit = row['unit_name']
    temp_df = water_df.loc[(water_df['analysis_name'] == crop) & (water_df['chemical_name'] == chemical) & (water_df['unit_name'] == unit)]
    df = pd.concat([df, temp_df])

In [35]:
df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
277,230,230,CO006WA0079,2012-07-19 00:00:00.000,Irrigation,sar,1.570,nan
359,312,312,CP052WA0005,2012-07-13 00:00:00.000,Irrigation,sar,1.090,nan
367,320,320,CP052WA0006,2012-07-13 00:00:00.000,Irrigation,sar,0.050,nan
382,335,335,CP052WA0007,2012-07-13 00:00:00.000,Irrigation,sar,1.280,nan
404,357,357,CP052WA0008,2012-07-13 00:00:00.000,Irrigation,sar,1.730,nan
...,...,...,...,...,...,...,...,...
656770,446831,446831,CU014WA0017,2020-04-09 10:34:00.000,KS Drinking Water Standard Analysis,nitrite,0.005,ppm
657779,447540,447540,CU014WA0019,2020-04-09 11:01:00.000,KS Drinking Water Standard Analysis,nitrite,0.005,ppm
699950,476832,476832,CB210WA0001,2020-05-12 10:49:00.000,KS Drinking Water Standard Analysis,nitrite,0.003,ppm
701452,477818,477818,CB210WA0002,2020-05-12 10:49:00.000,KS Drinking Water Standard Analysis,nitrite,0.003,ppm


In [37]:
df.to_csv("output/water_analysis_cleaned.csv",chunksize=1000)

In [38]:
df[["analysis_name", "chemical_name", "unit_name"]].set_index('analysis_name').to_csv('output/expected_units/analysis_units.csv')

In [39]:
# df[["analysis_name", "chemical_name", "unit_name"]].set_index('analysis_name').to_dict('dict') 

In [40]:
water_df = pd.read_csv("output/water_analysis_cleaned.csv")

In [41]:
water_df = water_df[['sample_code','analysis_name','chemical_name', 'result']]

In [42]:
water_df['analysis_name'] = [ i.split(",") for i in water_df['analysis_name'] ]

In [43]:
water_df['analysis_name']

0                                   [Irrigation]
1                                   [Irrigation]
2                                   [Irrigation]
3                                   [Irrigation]
4                                   [Irrigation]
                           ...                  
1006750    [KS Drinking Water Standard Analysis]
1006751    [KS Drinking Water Standard Analysis]
1006752    [KS Drinking Water Standard Analysis]
1006753    [KS Drinking Water Standard Analysis]
1006754    [KS Drinking Water Standard Analysis]
Name: analysis_name, Length: 1006755, dtype: object

In [45]:
water_df.analysis_name = [ i[0] for i in water_df.analysis_name ] 

In [48]:
water_df.chemical_name.value_counts()

bicarbonate               48654
ammonium                  48458
zinc                      46425
copper                    46419
boron                     46418
                          ...  
dissolved_silica             24
total_viable_count_@37       17
sodium_chloride               4
escherichia_coli              2
moisture                      2
Name: chemical_name, Length: 70, dtype: int64

In [46]:
df.to_csv("output/water_analysis_cleaned.csv",chunksize=1000)

In [ ]:
# _df = pd.DataFrame()
# for index, row in water_df.iterrows():
#     analyses = row['analysis_name']
#     sample_code = row['sample_code']
#     for analysis in analyses:
#         _ = pd.DataFrame(row).T
#         _['analysis_name'] = analysis
#         _df = pd.concat([_df, _])

# _df.to_csv("output/water_single_analysis.csv")

In [36]:
water_df = pd.read_csv("output/water_analysis_cleaned.csv")

In [37]:
water_df_pivoted = pd.pivot_table(data=water_df, index="sample_code", columns="chemical_name", values="result")

In [39]:
water_df_pivoted.columns

Index(['aluminium', 'ammonium', 'arsenic', 'barium', 'bicarbonate', 'bod',
       'boron', 'cadmium', 'calcium', 'carbonate', 'charge_balance',
       'chloride', 'chlorides', 'chromium', 'cobalt', 'cod',
       'colloidal_silica', 'colour', 'copper', 'dissolved_silica', 'e_coli',
       'electrical_conductivity', 'escherichia_coli', 'faecal_e_coli',
       'feacal_coliforms', 'feacal_coliforms_mpn', 'feacal_e_coli',
       'fluorides', 'hardness', 'iodine', 'iron', 'lead', 'magnesium',
       'manganese', 'mercury', 'moisture', 'molybdenum', 'nickel', 'nitrate_n',
       'nitrates', 'nitrite', 'oil_&_grease', 'ph', 'phosphate', 'phosphorus',
       'potassium', 'pseudomonas', 'salmonella', 'sar', 'selenium', 'shigella',
       'silica', 'silicon__', 'silver', 'sodium', 'sodium_chloride',
       'strontium', 'sulphate', 'sulphates', 'sulphur', 'tbc_at_22_degrees',
       'tbc_at_37_degrees', 'total_coliforms', 'total_dissolved_solids',
       'total_suspended_solids', 'total_viable_cou

In [43]:
water_df_pivoted.total_suspended_solids.info()

<class 'pandas.core.series.Series'>
Index: 20522 entries, AGD102-124WA0001 to I48-1-9WA0001
Series name: total_suspended_solids
Non-Null Count  Dtype  
--------------  -----  
2564 non-null   float64
dtypes: float64(1)
memory usage: 836.7+ KB
